<a href="https://colab.research.google.com/github/Rogerio-mack/IA_2022S2/blob/main/AI_A1_Lab_solucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right"> 
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->

<br>
<br>
<br>
<br>
<br>

# A1 Lab Caso: **Controle de Qualidade de Peças de uma Indústria** 
---

**pieces** é um data set para o controle de qualidade de peças produzidas por uma indústria. São empregadas quatro medidas (A, B, C e D) para o controle da qualidade das peças. As peças são então *Accept, Refurbish* ou *Reject* segundo o controle de qualidade (atributo `Quality`). A indústria ainda conta com 3 unidades para a produção dessas peças (atributo `Unit`). 

Seu trabalho é criar modelos supervisionados de classificação para predição da qualidade das peças e empregar o melhor modelo obtido (maior acuracidade) na predição da qualidade de novos casos. Em seguida você empregará um modelo de regressão linear para predição de medidas das peças.

<br>

***

<br>

Empregue os arquivos e crie os modelos conforme o final do seu TIA:

* **TIA PAR**: Modelos: **Knn k=7** e **Regressão Logística**, Arquivos:

>> Arquivo de Exemplos: http://meusite.mackenzie.br/rogerio/TIC/pieces3.csv

>> Arquivo de Novos Casos: http://meusite.mackenzie.br/rogerio/TIC/pieces3_new.csv

* **TIA ÍMPAR**: Modelos: **Knn k=9** e  **Regressão Logística**, Arquivos:

>> Arquivo de Exemplos: http://meusite.mackenzie.br/rogerio/TIC/pieces4.csv

>> Arquivo de Novos Casos: http://meusite.mackenzie.br/rogerio/TIC/pieces4_new.csv

Nos modelos não empregue nenhum ou parâmetro além dos informados.


<br>

***


# Q0. Aquisição dos Dados

In [ ]:
import pandas as pd

df = pd.read_csv('http://meusite.mackenzie.br/rogerio/TIC/pieces3.csv')
df.head()

,id,A,B,C,D,Quality,Unit
0,168,5.0,3.5,1.3,0.3,Reject,BH
1,527,6.9,3.1,5.4,2.1,Refurbish,BH
2,493,6.8,3.0,5.5,2.1,Refurbish,SP
3,584,5.9,3.0,5.1,1.8,Refurbish,SP
4,534,4.9,NaN,3.3,1.0,Accept,RJ


# Q1. Exploração e Preparação dos Dados

1. Explore seu data set.
2. Se houverem, elimine atributos que não devem fazer parte do modelo (treinamento).
3. Se existirem valores nulos elimine as linhas desses casos.
4. Faça o hot encode do atributo categórico `Unit` para o uso nos modelos. Em seguida exclua o atributo `Unit` original.
5. NÃO NORMALIZE dados ou faça outras transformações não solicitadas (troca da ordem de atributos por exemplo não são recomendadas!).





In [ ]:
df = df.drop(columns=['id'])
df.head()

,A,B,C,D,Quality,Unit
0,5.0,3.5,1.3,0.3,Reject,BH
1,6.9,3.1,5.4,2.1,Refurbish,BH
2,6.8,3.0,5.5,2.1,Refurbish,SP
3,5.9,3.0,5.1,1.8,Refurbish,SP
4,4.9,NaN,3.3,1.0,Accept,RJ


In [ ]:
df.isnull().sum()

A           0
B          12
C           3
D           3
Quality     0
Unit        0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
hot_code = pd.get_dummies(df.Unit, prefix_sep='_')
df = pd.concat([df,hot_code],axis=1)
df = df.drop(columns=['Unit'])
df.head()

,A,B,C,D,Quality,BH,RJ,SP
0,5.0,3.5,1.3,0.3,Reject,1,0,0
1,6.9,3.1,5.4,2.1,Refurbish,1,0,0
2,6.8,3.0,5.5,2.1,Refurbish,0,0,1
3,5.9,3.0,5.1,1.8,Refurbish,0,0,1
5,5.5,2.3,4.0,1.3,Accept,0,1,0


In [ ]:
print('Nr de linhas e colunas após as transformações: ', df.shape)

Nr de linhas e colunas após as transformações:  (562, 8)


In [ ]:
print('Frequencias do atributo Quality:\n', df.Quality.value_counts())

Frequencias do atributo Quality:
 Refurbish    176
Reject       156
Accept       149
Name: Quality, dtype: int64


# Q2-Q3. Crie e Avalie os diferentes Modelos de Classificação

Crie os modelos conforme o final do seu TIA. Empregue o modelo de maior acurácia para predição dos novos casos. Em todos casos:

1. Empregue o `seed=1984` para a geração dos dados de treinamento e teste e para o `random_state` das Árvores de Decisão.
2. Crie os conjuntos de treinamento e teste com 0.3 para teste e dados estratificados.
3. Não empregue outros parâmetros não solicitados nos modelos.
4. Para os modelos de regressão logística ignore eventuais comentários, warnings produzidos pelo modelo.



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 1. Variáveis preditoras e objetivo
X = df[['A', 'B', 'C', 'D', 'BH', 'RJ', 'SP']]    
y = df[['Quality']]  

# 2. Separação Treinamento e Teste
seed = 1984
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=seed)

# 3. Definição do Modelo
models = [ KNeighborsClassifier(7), LogisticRegression() ]

# 4. Treinamento
for clf in models:
  clf.fit(X_train, y_train)                  

# 5. Avaliação sobre o conjunto de Teste
acc = []
for clf in models:
  y_pred = clf.predict(X_test)
  
  print(120*'*','\n')
  accuracy = accuracy_score(y_test, y_pred)
  print(clf, '\n', accuracy)

  acc.append(accuracy)
  
  cm = confusion_matrix(y_test, y_pred)
  print('\n', cm)
    
  print('\n', classification_report(y_test, y_pred))



************************************************************************************************************************ 

KNeighborsClassifier(n_neighbors=7) 
 0.9408284023668639

 [[48  3  0]
 [ 7 51  0]
 [ 0  0 60]]

               precision    recall  f1-score   support

      Accept       0.87      0.94      0.91        51
   Refurbish       0.94      0.88      0.91        58
      Reject       1.00      1.00      1.00        60

    accuracy                           0.94       169
   macro avg       0.94      0.94      0.94       169
weighted avg       0.94      0.94      0.94       169

************************************************************************************************************************ 

LogisticRegression() 
 0.9881656804733728

 [[49  2  0]
 [ 0 58  0]
 [ 0  0 60]]

               precision    recall  f1-score   support

      Accept       1.00      0.96      0.98        51
   Refurbish       0.97      1.00      0.98        58
      Reject       1.00      

/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_i

# Q4. Predição com o Melhor Modelo (Acurácia Geral)

In [ ]:
X_new = pd.read_csv('http://meusite.mackenzie.br/rogerio/TIC/pieces3_new.csv')
display(X_new.head())

hot_code = pd.get_dummies(X_new.Unit, prefix_sep='_')
X_new = pd.concat([X_new,hot_code],axis=1)
X_new = X_new.drop(columns=['Unit'])
X_new.head() 

,A,B,C,D,Unit
0,6.1,2.8,4.7,1.2,BH
1,5.5,2.6,4.4,1.2,SP
2,7.7,3.0,6.1,2.3,BH
3,7.2,3.2,6.0,1.8,BH
4,4.6,3.4,1.4,0.3,RJ


,A,B,C,D,BH,RJ,SP
0,6.1,2.8,4.7,1.2,1,0,0
1,5.5,2.6,4.4,1.2,0,0,1
2,7.7,3.0,6.1,2.3,1,0,0
3,7.2,3.2,6.0,1.8,1,0,0
4,4.6,3.4,1.4,0.3,0,1,0


In [ ]:
# 6. Predição de Novos casos                   
clf = models[acc.index(max(acc))]                # Escolha o Melhor modelo... 

print( clf )
print( clf.predict(X_new) )

LogisticRegression()
['Accept' 'Accept' 'Refurbish' 'Refurbish' 'Reject' 'Reject']


# Q5. Regressão Linear

Crie um modelo de Regressão Linear para o Cálculo da medida C das peças com base nas demais medidas. Empregue o modelo para estimar a medida dos C das novas peças.

In [ ]:
import statsmodels.formula.api as sm

model = sm.ols(formula='C ~ A + B + D', data=df)
result = model.fit()

y_pred = result.predict(X_new[['A','B','D']])

print(result.summary())
print(y_pred)



                            OLS Regression Results                            
Dep. Variable:                      C   R-squared:                       0.970
Model:                            OLS   Adj. R-squared:                  0.970
Method:                 Least Squares   F-statistic:                     6111.
Date:                Mon, 12 Dec 2022   Prob (F-statistic):               0.00
Time:                        13:21:29   Log-Likelihood:                -143.35
No. Observations:                 562   AIC:                             294.7
Df Residuals:                     558   BIC:                             312.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.8185      0.148     -5.535      0.0